In [3]:
#这个方法中必须要输入一个元组作为参数。 如果你恰巧有一个 list 或者 set 类型的选择项，
#要确保传递参数前先调用 tuple() 将其转换为元组类型。


## 2.3 用Shell通配符匹配字符串


### 问题


你想使用 Unix Shell 中常用的通配符(比如 *.py , Dat[0-9]*.csv 等)去匹配文本字符串

### 解决方案


fnmatch 模块提供了两个函数—— fnmatch() 和 fnmatchcase() ，可以用来实现这样的匹配。用法如下：

In [4]:
from fnmatch import fnmatch, fnmatchcase
fnmatch('foo.txt', '*.txt')

True

In [ ]:
fnmatch('foo.txt', '?oo.txt')

In [ ]:
fnmatch('Dat45.csv', 'Dat[0-9]*')

In [5]:
names = ['Dat1.csv', 'Dat2.csv', 'config.ini', 'foo.py']
[name for name in names if fnmatch(name, 'Dat*.csv')]

['Dat1.csv', 'Dat2.csv']

fnmatch() 函数使用底层操作系统的大小写敏感规则(不同的系统是不一样的)来匹配模式。
如果你对这个区别很在意，可以使用 fnmatchcase() 来代替。它完全使用你的模式大小写匹配。比如：

In [ ]:
fnmatchcase('foo.txt', '*.TXT')

In [12]:
# Search for the location of the first occurrence
text = 'yeah, but no, but yeah, but no, but yeah'
text.find('no')

10

对于复杂的匹配需要使用正则表达式和 re 模块。
为了解释正则表达式的基本原理，假设你想匹配数字格式的日期字符串比如 11/27/2012 ，你可以这样做：

In [14]:
text1 = '11/27/2012'
text2 = 'Nov 27, 2012'
import re
# Simple matching: \d+ means match one or more digits
if re.match(r'\d+/\d+/\d+', text2):
    print('yes')
else:
    print('no')

no


match() 总是从字符串开始去匹配，如果你想查找字符串任意部分的模式出现位置，
使用 findall() 方法去代替。比如：



findall() 方法会搜索文本并以列表形式返回所有的匹配。
如果你想以迭代方式返回匹配，可以使用 finditer() 方法来代替

In [37]:
from calendar import month_abbr
datepat = re.compile(r'(\d+)/(\d+)/(\d+)')
text = 'Today is 11/27/2012. PyCon starts 3/13/2013.'
def change_date(m):
    mon_name = month_abbr[int(m.group(1))]
    return '{} {} {}'.format(m.group(2), mon_name, m.group(3))
datepat.sub(change_date, text)

'Today is 27 Nov 2012. PyCon starts 13 Mar 2013.'

In [38]:
def matchcase(word):
    def replace(m):
        text = m.group()
        if text.isupper():
            return word.upper()
        elif text.islower():
            return word.lower()
        elif text[0].isupper():
            return word.capitalize()
        else:
            return word
    return replace

下面是使用上述函数的方法：

In [39]:
text = 'UPPER PYTHON, lower python, Mixed Python'

re.sub('python', matchcase('snake'), text, flags=re.IGNORECASE)

'UPPER SNAKE, lower snake, Mixed Snake'

你正在试着用正则表达式匹配某个文本模式，但是它找到的是模式的最长可能匹配。
而你想修改它变成查找最短的可能匹配。

In [53]:
s = 'pýtĥöñ\fis\tawesome\r\n'
s

'pýtĥöñ\x0cis\tawesome\r\n'

第一步是清理空白字符。为了这样做，先创建一个小的转换表格然后使用 translate() 方法：

In [54]:
remap = {
    ord('\t') : ' ',
    ord('\f') : ' ',
    ord('\r') : None # Deleted
}
a = s.translate(remap)
a

'pýtĥöñ is awesome\n'

正如你看的那样，空白字符 \t 和 \f 已经被重新映射到一个空格。回车字符r直接被删除。

你可以以这个表格为基础进一步构建更大的表格。比如，让我们删除所有的和音符：

In [ ]:
import unicodedata
import sys
cmb_chrs = dict.fromkeys(c for c in range(sys.maxunicode)
                        if unicodedata.combining(chr(c)))
b = unicodedata.normalize('NFD', a)
b

In [ ]:
b.translate(cmb_chrs)

上面例子中，通过使用 dict.fromkeys() 方法构造一个字典，每个Unicode和音符作为键，对应的值全部为 None 。

然后使用 unicodedata.normalize() 将原始输入标准化为分解形式字符。
然后再调用 translate 函数删除所有重音符。
同样的技术也可以被用来删除其他类型的字符(比如控制字符等)。

作为另一个例子，这里构造一个将所有Unicode数字字符映射到对应的ASCII字符上的表格：

In [ ]:
digitmap = { c: ord('0') + unicodedata.digit(chr(c))
        for c in range(sys.maxunicode)
        if unicodedata.category(chr(c)) == 'Nd' }
len(digitmap)

In [ ]:
# Arabic digits
x = '\u0661\u0662\u0663'
x.translate(digitmap)

另一种清理文本的技术涉及到I/O解码与编码函数。这里的思路是先对文本做一些初步的清理，
然后再结合 encode() 或者 decode() 操作来清除或修改它。比如：

In [ ]:
a

In [ ]:
b = unicodedata.normalize('NFD', a)
b.encode('ascii', 'ignore').decode('ascii')

这里的标准化操作将原来的文本分解为单独的和音符。接下来的ASCII编码/解码只是简单的一下子丢弃掉那些字符。
当然，这种方法仅仅只在最后的目标就是获取到文本对应ACSII表示的时候生效。

这里的关键点在于原始的生成器函数并不需要知道使用细节，它只负责生成字符串片段就行了。

使用 textwrap 模块来格式化字符串的输出。比如，假如你有下列的长字符串：

In [55]:
s = "Look into my eyes, look into my eyes, the eyes, the eyes, \
the eyes, not around the eyes, don't look around the eyes, \
look into my eyes, you're under."

下面演示使用 textwrap 格式化字符串的多种方式：

In [56]:
textwrap 模块对于字符串打印是非常有用的，特别是当你希望输出自动匹配终端大小的时候
import textwrap
print(textwrap.fill(s, 70))

Look into my eyes, look into my eyes, the eyes, the eyes, the eyes,
not around the eyes, don't look around the eyes, look into my eyes,
you're under.


In [69]:


你想将HTML或者XML实体如 &entity; 或 &#code; 替换为对应的文本。
再者，你需要转换文本中特定的字符(比如<, >, 或 &)。

### 解决方案


如果你想替换文本字符串中的 ‘<’ 或者 ‘>’ ，使用 html.escape() 函数可以很容易的完成。比如：

s = 'Elements are written as "<tag>text</tag>".'
import html
print(s)

print(html.escape(s))

# Disable escaping of quotes
print(html.escape(s, quote=False))

SyntaxError: invalid syntax (<ipython-input-69-f41ccc54b955>, line 1)

In [ ]:
#有一个字符串，想从左至右将其解析为一个令牌流。
text = 'foo = 23 + 42 * 10'

为了令牌化字符串，你不仅需要匹配模式，还得指定模式的类型。
比如，你可能想将字符串像下面这样转换为序列对：

In [ ]:
tokens = [('NAME', 'foo'), ('EQ','='), ('NUM', '23'), ('PLUS','+'),
          ('NUM', '42'), ('TIMES', '*'), ('NUM', '10')]

为了执行这样的切分，第一步就是像下面这样利用命名捕获组的正则表达式来定义所有可能的令牌，包括空格：

In [ ]:
import re
NAME = r'(?P<NAME>[a-zA-Z_][a-zA-Z_0-9]*)'
NUM = r'(?P<NUM>\d+)'
PLUS = r'(?P<PLUS>\+)'
TIMES = r'(?P<TIMES>\*)'
EQ = r'(?P<EQ>=)'
WS = r'(?P<WS>\s+)'

master_pat = re.compile('|'.join([NAME, NUM, PLUS, TIMES, EQ, WS]))

在上面的模式中， ?P<TOKENNAME> 用于给一个模式命名，供后面使用。

下一步，为了令牌化，使用模式对象很少被人知道的 scanner() 方法。
这个方法会创建一个 scanner 对象，
在这个对象上不断的调用 match() 方法会一步步的扫描目标文本，每步一个匹配。
下面是演示一个 scanner 对象如何工作的交互式例子：

In [ ]:
scanner = master_pat.scanner('foo = 42')
scanner.match()

In [ ]:
_.lastgroup, _.group()

In [ ]:
scanner.match()

In [ ]:
_.lastgroup, _.group()

In [ ]:
scanner.match()

In [ ]:
_.lastgroup, _.group()

In [ ]:
scanner.match()

In [ ]:
_.lastgroup, _.group()

In [ ]:
scanner.match()

In [ ]:
_.lastgroup, _.group()

In [ ]:
scanner.match()

实际使用这种技术的时候，可以很容易的像下面这样将上述代码打包到一个生成器中：

In [ ]:
def generate_tokens(pat, text):
    Token = namedtuple('Token', ['type', 'value'])
    scanner = pat.scanner(text)
    for m in iter(scanner.match, None):
        yield Token(m.lastgroup, m.group())

# Example use
for tok in generate_tokens(master_pat, 'foo = 42'):
    print(tok)
# Produces output
# Token(type='NAME', value='foo')
# Token(type='WS', value=' ')
# Token(type='EQ', value='=')
# Token(type='WS', value=' ')
# Token(type='NUM', value='42')

如果你想过滤令牌流，你可以定义更多的生成器函数或者使用一个生成器表达式。
比如，下面演示怎样过滤所有的空白令牌：

In [ ]:
tokens = (tok for tok in generate_tokens(master_pat, text)
          if tok.type != 'WS')
for tok in tokens:
    print(tok)

### 讨论


通常来讲令牌化是很多高级文本解析与处理的第一步。
为了使用上面的扫描方法，你需要记住这里一些重要的几点。
第一点就是你必须确认你使用正则表达式指定了所有输入中可能出现的文本序列。
如果有任何不可匹配的文本出现了，扫描就会直接停止。这也是为什么上面例子中必须指定空白字符令牌的原因。

令牌的顺序也是有影响的。 re 模块会按照指定好的顺序去做匹配。
因此，如果一个模式恰好是另一个更长模式的子字符串，那么你需要确定长模式写在前面。比如：

In [ ]:
LT = r'(?P<LT><)'
LE = r'(?P<LE><=)'
EQ = r'(?P<EQ>=)'

master_pat = re.compile('|'.join([LE, LT, EQ])) # Correct
# master_pat = re.compile('|'.join([LT, LE, EQ])) # Incorrect

第二个模式是错的，因为它会将文本<=匹配为令牌LT紧跟着EQ，而不是单独的令牌LE，这个并不是我们想要的结果。

最后，你需要留意下子字符串形式的模式。比如，假设你有如下两个模式：

In [ ]:
PRINT = r'(?P<PRINT>print)'
NAME = r'(?P<NAME>[a-zA-Z_][a-zA-Z_0-9]*)'

master_pat = re.compile('|'.join([PRINT, NAME]))

for tok in generate_tokens(master_pat, 'printer'):
    print(tok)

# Outputs :
# Token(type='PRINT', value='print')
# Token(type='NAME', value='er')

关于更高阶的令牌化技术，你可能需要查看 PyParsing
或者 PLY 包。
一个调用PLY的例子在下一节会有演示。

## 2.20 字节字符串上的字符串操作


### 问题


你想在字节字符串上执行普通的文本操作(比如移除，搜索和替换)。

### 解决方案


字节字符串同样也支持大部分和文本字符串一样的内置操作。比如：

In [66]:
data = b'Hello World'
data[0:5]

b'Hello'

In [67]:
data.startswith(b'Hello')

True

In [68]:
data.split()

[b'Hello', b'World']

In [ ]:
data.replace(b'Hello', b'Hello Cruel')

这些操作同样也适用于字节数组。比如：

In [ ]:
data = bytearray(b'Hello World')
data[0:5]

In [ ]:
data.startswith(b'Hello')

In [ ]:
data.split()

In [ ]:
data.replace(b'Hello', b'Hello Cruel')

你可以使用正则表达式匹配字节字符串，但是正则表达式本身必须也是字节串。比如：

In [ ]:
data = b'FOO:BAR,SPAM'
import re
re.split('[:,]',data)

In [ ]:
re.split(b'[:,]',data) # Notice: pattern as bytes

### 讨论


大多数情况下，在文本字符串上的操作均可用于字节字符串。
然而，这里也有一些需要注意的不同点。首先，字节字符串的索引操作返回整数而不是单独字符。比如：

In [ ]:
a = 'Hello World' # Text string
a[0]

In [ ]:
a[1]

In [ ]:
b = b'Hello World' # Byte string
b[0]

In [ ]:
b[1]

这种语义上的区别会对于处理面向字节的字符数据有影响。

第二点，字节字符串不会提供一个美观的字符串表示，也不能很好的打印出来，除非它们先被解码为一个文本字符串。比如：

In [ ]:
s = b'Hello World'
print(s)

In [ ]:
print(s.decode('ascii'))

类似的，也不存在任何适用于字节字符串的格式化操作：

In [ ]:
b'%10s %10d %10.2f' % (b'ACME', 100, 490.1)

In [ ]:
b'{} {} {}'.format(b'ACME', 100, 490.1)

如果你想格式化字节字符串，你得先使用标准的文本字符串，然后将其编码为字节字符串。比如：

In [ ]:
'{:10s} {:10d} {:10.2f}'.format('ACME', 100, 490.1).encode('ascii')

最后需要注意的是，使用字节字符串可能会改变一些操作的语义，特别是那些跟文件系统有关的操作。
比如，如果你使用一个编码为字节的文件名，而不是一个普通的文本字符串，会禁用文件名的编码/解码。比如：

In [ ]:
# Write a UTF-8 filename
with open('jalape\xf1o.txt', 'w') as f:
    f.write('spicy')
# Get a directory listing
import os
os.listdir('.') # Text string (names are decoded)

In [ ]:
os.listdir(b'.') # Byte string (names left as bytes)

注意例子中的最后部分给目录名传递一个字节字符串是怎样导致结果中文件名以未解码字节返回的。
在目录中的文件名包含原始的UTF-8编码。
参考5.15小节获取更多文件名相关的内容。

最后提一点，一些程序员为了提升程序执行的速度会倾向于使用字节字符串而不是文本字符串。
尽管操作字节字符串确实会比文本更加高效(因为处理文本固有的Unicode相关开销)。
这样做通常会导致非常杂乱的代码。你会经常发现字节字符串并不能和Python的其他部分工作的很好，
并且你还得手动处理所有的编码/解码操作。
坦白讲，如果你在处理文本的话，就直接在程序中使用普通的文本字符串而不是字节字符串。不做死就不会死！